<a href="https://colab.research.google.com/github/lanchuhuong/Food-Recognition/blob/main/Visiontransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install -q torchbearer

In [ ]:
!pip uninstall -y kaggle #need this to use kaggle api
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [ ]:

!mkdir ~/.kaggle
!echo '{"username":"lanchu","key":"004e7e514e204a75f54db6f58796d439"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c food-recognition-challenge-2021 -p data

In [ ]:
!unzip -q data/food-recognition-challenge-2021.zip -d data

replace data/sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
pd.set_option('display.max_columns', 50)
import re
import timeit
import random
random.seed(3)
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.metrics import r2_score
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import skimage.transform

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torchbearer
from torchbearer.callbacks import imaging
from torchvision import transforms
from torchbearer.cv_utils import DatasetValidationSplitter
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks import imaging
from transformers import AutoFeatureExtractor, AutoModel


PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [ ]:
labels = pd.read_csv('data/train_labels.csv').set_index('img_name')
labels = labels.to_dict()
labels = labels[list(labels.keys())[0]]
print(f'lenght of dictionary: {len(labels)}')
df = pd.DataFrame(list(labels.items()))
df.columns = ['images', 'labels']
df = df.astype({'labels': str})
# Display the occurance of each class
df.groupby('labels').count()

lenght of dictionary: 30612


,images
labels,
1,474
10,461
11,392
12,417
13,286
...,...
78,274
79,421
8,409


In [ ]:
import os
from shutil import copyfile

os.makedirs("data/train", exist_ok=True)
# os.makedirs("data/val", exist_ok=True)

for label in df['labels'].unique():
  os.makedirs(f"data/train/{label}", exist_ok=True)
  # os.makedirs(f"data/val/{label}", exist_ok=True)

for image, label in df.values:
  copyfile(f"data/train_set/train_set/{image}", f"data/train/{label}/{image}")
  # copyfile(f"data/val_set/val_set/{image}", f"data/val/{label}/{image}")


In [ ]:
input_size = 224
batch_size = 32
#Data augmentation:
#1.Flipping: flip each image horizontally by a chance of 50% (transforms.RandomHorizontalFlip).
#2. transforms.RandomResizedCrop. This transformation scales the image in a small range, while eventually changing the aspect ratio, and crops it afterward in the previous size. Therefore, the actual pixel values change while the content or overall semantics of the image stays the same.
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomResizedCrop((32,32),scale=(0.8,1.0),ratio=(0.9,1.1)),
                                transforms.ToTensor(), transforms.RandomResizedCrop(input_size)])#resize the image to the input size, the model was trained on size 244*244

train_data = datasets.ImageFolder('data/train', transform=transform)
train_data, val_data = torch.utils.data.random_split(train_data,[25000,5612])

trainset = torch.utils.data.DataLoader(train_data, pin_memory=True, batch_size=batch_size, shuffle=True)
valset = torch.utils.data.DataLoader(val_data, pin_memory=True, batch_size=batch_size, shuffle=True)

In [ ]:
from tqdm import tqdm
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForImageClassification.from_pretrained("nateraw/food")

for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Linear(768, 80)
model.to(device)

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)
loss_fun = nn.CrossEntropyLoss()

for epoch in range(5):  # loop over the dataset multiple times
    model.train()
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
    for ttype, dataset in zip(["train", "val"], [trainset, valset]):
      running_loss = 0.0
      correct_values = 0.0
      count = 0.0
      for i, data in tqdm(enumerate(dataset)):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)

          # zero the parameter gradients
          if ttype=='train':
            optimizer.zero_grad()

          outputs = model(inputs)
          loss = loss_fun(outputs['logits'], labels)
          if ttype=='train':
            loss.backward()
            optimizer.step()
          # pred = torch.max(outputs.data, 1)
          #correct_values = (pred == labels).sum()
          correct_values += (outputs == labels).float().sum()
          #correct_values += (outputs.argmax(dim=-1) == labels).sum().item()
          count += labels.shape[0]
          running_loss += loss.item
      #loss = running_loss / len(dataset)
      loss = running_loss / count
      #accuracy = correct_values / len(dataset)
      accuracy = correct_values / count
      if ttype == 'train':
        train_acc.append(accuracy)
        train_loss.append(loss)
      if ttype == 'val':
        val_acc.append(accuracy)
        val_loss.append(loss)

      print(f"epoch {epoch}: train_acc = {train_acc} | train_loss = {train_loss} | val_acc = {val_acc} | val_loss = {val_loss}")
# print('Finished Training')

0it [00:22, ?it/s]


AttributeError: ignored

In [ ]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')